In [ ]:
# default_exp train

# Hemmaplan
 > This is where we train our models, using our "Home Court" advantage.

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

## Data

In [ ]:
#exports
import numpy as np
import torch
from torch.utils.data import Dataset


class ChessValueDataset(Dataset):
    """Chess games dataset"""
    def __init__(self):
        dat = np.load("processed/dataset_1k.npz")
        self.X = dat["arr_0"]
        self.Y = dat["arr_1"]
        print("loaded", self.X.shape, self.Y.shape)
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return (self.X[idx], self.Y[idx])

In [ ]:
chess_dataset = ChessValueDataset()

loaded (1140, 5, 8, 8) (1140,)


In [ ]:
# Divides the data into mini batches.
train_loader = torch.utils.data.DataLoader(chess_dataset, batch_size=1024, shuffle=True)

## Neural Network
> Convolutionl deep neural network that trains on our chess data. 

In [ ]:
#exports
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # First convolutional layer taking in the 5 chanels for board state,
        # outputting 16 convolutional features, with a square kernel size of 3
        self.a1 = nn.Conv2d(5, 16, kernel_size=3, padding=1)                 
        self.a2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.a3 = nn.Conv2d(16, 32, kernel_size=3, stride=2)

        self.b1 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.b2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.b3 = nn.Conv2d(32, 64, kernel_size=3, stride=2)

        self.c1 = nn.Conv2d(64, 64, kernel_size=2, padding=1)
        self.c2 = nn.Conv2d(64, 64, kernel_size=2, padding=1)
        self.c3 = nn.Conv2d(64, 128, kernel_size=2, stride=2)

        self.d1 = nn.Conv2d(128, 128, kernel_size=1)
        self.d2 = nn.Conv2d(128, 128, kernel_size=1)
        self.d3 = nn.Conv2d(128, 128, kernel_size=1)

        self.last = nn.Linear(128, 1)

    # x represents our data
    def forward(self, x):
        # Pass data through the first convolutional layer
        # and apply the rectified-linear activation function over x
        x = F.relu(self.a1(x))
        x = F.relu(self.a2(x))
        x = F.relu(self.a3(x))

        # Run max pooling over x to down-sample with kernel size of 2
        # x = F.max_pool2d(x, 2)

        # 4x4
        x = F.relu(self.b1(x))
        x = F.relu(self.b2(x))
        x = F.relu(self.b3(x))

        # 2x2 
        x = F.relu(self.c1(x))
        x = F.relu(self.c2(x))
        x = F.relu(self.c3(x))

        # 1x128

        x = F.relu(self.d1(x))
        x = F.relu(self.d2(x))
        x = F.relu(self.d3(x))

        x = x.view(-1, 128)
        x = self.last(x)

        # value output
        return torch.tanh(x)      

In [ ]:
# Creates a Neural Network according from our Net class with Adam as optimizer.
model = Net()
optimizer = torch.optim.Adam(model.parameters())

# Use choose between cpu and cuda
device = "cpu"
if device == "cuda":
    model.cuda()

# Sets the model in training mode
model.train();

criterion = nn.MSELoss()

for epoch in range(10): # loop over the dataset multiple times

    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        target = target.unsqueeze(-1)
        data = data.float()
        target = target.float()
        data, target = data.to(device), target.to(device)

        # Resets the gradients in the optimizer to zero.
        optimizer.zero_grad()

        # Pass data through our model.
        output = model(data)

        # Calculate loss between our outputs and targets.
        loss = criterion(output, target)
        # Computes the gradient of current tensor w.r.t graph leaves. 
        loss.backward() 
        # Performs a parameter update to our model.
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if batch_idx % 200 == 199:    # print every 199 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')
# last model trained for 10 epochs on 10M data 0.547 loss
#torch.save(model.state_dict(), "nets/value_gen2.pth")

/home/richard/miniconda3/envs/fastai2/lib/python3.8/site-packages/torch/autograd/__init__.py:147: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/cuda/CUDAFunctions.cpp:115.)
  Variable._execution_engine.run_backward(


Finished Training


In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_state.ipynb.
Converted 01_generate_training_set.ipynb.
Converted 02_train.ipynb.
Converted 03_play.ipynb.
Converted index.ipynb.
